# 二维码实时识别

功能描述：能够实时识别摄像头画面中的二维码信息。

准备工作：摄像头（任何一款普通的免驱摄像头）

代码编写：谢作如

## 基本库

摄像头使用OpenCV，二维码的库选择了ZXing。这是一个支持在图像中解码和生成条形码(如二维码、PDF 417、EAN、UPC、Aztec、Data Matrix、Codabar)的库。ZXing(“zebra crossing”)是一个开源的、多格式的、用Java实现的一维/二维条码图像处理库，具有到其他语言的端口。

In [ ]:
# 应用摄像头的准备工作
# -*- coding: UTF-8 -*-
import cv2
import zxing
import time
from IPython import display
from matplotlib import pyplot as plt
%matplotlib inline

## 核心函数

请根据需要，运行相应的函数，也可以全部运行。

In [ ]:
#基本函数1：获取摄像头内容保存为图片
def getcampic(fname):
    cap = cv2.VideoCapture(0)        # 打开摄像头
    ret, frame = cap.read()       # 读摄像头
    cv2.imwrite(fname,frame)
    cap.release()      
    cv2.destroyAllWindows() 
    return True
getcampic("1.jpg")

In [ ]:
#基本函数2：在网页上显示摄像头图片
def showcampic():
    cap = cv2.VideoCapture(0)        # 打开摄像头
    ret, frame = cap.read()       # 读摄像头
    display.clear_output(wait=True) # 设置在一个画面中刷新
    plt.imshow(frame[:,:,::-1])
    plt.axis('off') #不显示坐标
    plt.show()
    cap.release()      
    cv2.destroyAllWindows() 

In [ ]:
#基本函数3：显示摄像头图片并且保存为图片文件
def get_showcampic(fname):
    cap = cv2.VideoCapture(0)        # 打开摄像头
    ret, frame = cap.read()       # 读摄像头
    display.clear_output(wait=True) # 设置在一个画面中刷新
    cv2.imwrite(fname,frame)
    plt.imshow(frame[:,:,::-1])
    plt.axis('off') #不显示坐标
    plt.show()
    cap.release()      
    cv2.destroyAllWindows() 

## zxing库的使用

拍照并识别图片中的二维码

In [ ]:
f="vv-text.png"
zx = zxing.BarCodeReader()
code = zx.decode(f)
if code is not None:
    print(code)
else:
    print("没有找到二维码")

返回的信息为BarCode对象，常用的属性有raw和type，分别表示识别内容和类型。类型分为TEXT、URI等。TEXT为文本，URI为网址。

In [ ]:
f="vv-url.png"
zx = zxing.BarCodeReader()
code = zx.decode(f)
if code is not None:
    print(code)
else:
    print("没有找到二维码")

In [ ]:
print("识别结果为：{s1}，类型为：{s2}".format(s1=code.raw,s2=code.type))

## 摄像头逐一识别

如何使用？只要将基本库导入和相应的核心函数一起放在自己的代码中，即可操作摄像头了。拍摄一张，识别一张。

In [ ]:
f="test.png"
get_showcampic(f)
zx = zxing.BarCodeReader()
code = zx.decode(f)
if code is not None:
    print("识别结果为：{s1}，类型为：{s2}".format(s1=code.raw,s2=code.type))
else:
    print("没有找到二维码")

## 摄像头实时识别

做一个不间断的循环，保存图片并识别，发现有二维码信息，则检查是否前面已经识别过的信息，如果不是则存入scv文件。

found是一个无序不重复元素集，避免重复识别。“found.clear()”语句的增加，仅仅为了避免同一张图片的重复识别，间隔一下，又可以识别了。为了确保识别效果，二维码的图片要大一点。

在使用中会发现摄像头存在比较严重的延时。网页显示本来就有点延时，加上缓冲区的延时，使用起来有点不习惯。代码中采用了两种方法，一是设置缓冲区为1帧，cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)；二是连续多次读取数据，让缓冲区尽快清空，ret,frame = cap.read()。现在估计还有2-3秒的延时。

注意：如果要再次启动，需要先停止“服务”。

In [ ]:
import cv2
import zxing
import time
from IPython import display
from matplotlib import pyplot as plt
%matplotlib inline
import csv
from PIL import Image
import threading

PATH = "test.csv"
img_name="test.jpg"
# 创建一个无序不重复元素集
found = set()
zx = zxing.BarCodeReader()
cap = cv2.VideoCapture(0)        # 打开摄像头
# 设置缓冲区为1帧
cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)
while(1):
    ret,frame = cap.read()
    ret,frame = cap.read()
    ret,frame = cap.read()
    #生成图片
    cv2.imwrite(img_name,frame)
    img = Image.open(img_name)
    display.clear_output(wait=True) # 设置在一个画面中刷新
    plt.imshow(img)
    plt.axis('off') #不显示坐标
    plt.show()
    code = zx.decode(img_name)
    if code is not None:
        s1=code.raw
        if s1 not in found:
            with open(PATH,'a+') as f:
                csv_write = csv.writer(f)
                csv_write.writerow(s1)
            # 先清除再增加，确保同一张图不会重复识别。
            found.clear()
            found.add(s1)
    if cv2.waitKey(1) == ord('q'):
        break